<a href="https://colab.research.google.com/github/sarraj-sareef/metaphor-detection-tamil/blob/main/Binary_classifier_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.

In [2]:
# pip install tensorflow

In [3]:
# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

# Hugging Face Dataset
from datasets import Dataset

# Model performance evaluation
import evaluate
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [4]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# from huggingface_hub import notebook_login

# notebook_login()

In [6]:
# def compute_metrics(p):
#     print(type(p))
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [7]:
# def tokenize_function(examples):
#     return tokenizer(examples["Lyrics"], padding="max_length", truncation=True)

In [8]:

df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/binary_classification.csv')
df.head()

,Song Name,Composer,Album,Lyricist,text,labels
0,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,அந்த கண்ண பார்த்தாக்கா லவ்வு தானா தோனாதா,0
1,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,அவன் கிட்ட போனாக்கா மனம் மானா மாறாதா,1
2,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,அகமெல்லாம் அவன்தான் அவன்தான் இருந்தான்,0
3,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,நடந்தால் அவன் கனவெல்லாமே அவன் முகம் தானே…ஏ….,0
4,அந்த கண்ண பார்த்தாக்கா,அனிருத் ரவிசந்தர்,மாஸ்டர்,விக்னேஷ் சிவன்,அழகன்தான் அவன்தான் அவன்தான் அழகா அளவா அவன் சிர...,0


In [9]:
df.drop(['Song Name', 'Composer', 'Album', 'Lyricist'], axis=1, inplace=True)

df.head()

,text,labels
0,அந்த கண்ண பார்த்தாக்கா லவ்வு தானா தோனாதா,0
1,அவன் கிட்ட போனாக்கா மனம் மானா மாறாதா,1
2,அகமெல்லாம் அவன்தான் அவன்தான் இருந்தான்,0
3,நடந்தால் அவன் கனவெல்லாமே அவன் முகம் தானே…ஏ….,0
4,அழகன்தான் அவன்தான் அவன்தான் அழகா அளவா அவன் சிர...,0


In [10]:
X=list(df['text'])

In [11]:
y=list(df['labels'])

In [12]:
import pandas as pd

train_data = df.sample(frac=0.8, random_state=42)  # 80% of the data for training
temp_data = df.drop(train_data.index)              # Remaining 20% of the data for further splitting

eval_data = temp_data.sample(frac=0.5, random_state=42)  # Splitting the remaining data into half for validation
test_data = temp_data.drop(eval_data.index)              # The rest goes to the test dataset

print(f'The training dataset has {len(train_data)} records.')
print(f'The evaluation dataset has {len(eval_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')


The training dataset has 741 records.
The evaluation dataset has 92 records.
The testing dataset has 93 records.


In [13]:
hg_train_data = Dataset.from_pandas(train_data)
hg_eval_data = Dataset.from_pandas(eval_data)
hg_test_data = Dataset.from_pandas(test_data)
print(hg_train_data)

Dataset({
    features: ['text', 'labels', '__index_level_0__'],
    num_rows: 741
})


In [14]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [15]:
from transformers import AutoTokenizer
# model = "monsoon-nlp/tamillion"
# model = "google/muril-base-cased"
model = "Ambareeshkumar/BERT-Tamil"
# model = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model)
# tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
# tokenizer = AutoTokenizer.from_pretrained("Ambareeshkumar/BERT-Tamil")
# tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/tamil-bert")

# tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/tamil-bert")
# tokenized_data = tokenizer(X_train, return_tensors="np", padding=True)
# # Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
# tokenized_data = dict(tokenized_data)

# labels = np.array(y_train)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [16]:
def tokenize_dataset(data):
    return tokenizer(data["text"],
                     max_length=32,
                     truncation=True,
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_eval = hg_eval_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/741 [00:00<?, ? examples/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

In [17]:
print(dataset_train)
print(dataset_eval)
print(dataset_test)

Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 741
})
Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 92
})
Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 93
})


In [18]:
from transformers import AutoModelForSequenceClassification
# from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = AutoModelForSequenceClassification.from_pretrained(model,num_labels=2,ignore_mismatched_sizes=True)
# Lower learning rates are often better for fine-tuning transformers
# model.compile(optimizer=Adam(3e-5))  # No loss argument!

# model.fit(tokenized_data, labels)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/709M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Ambareeshkumar/BERT-Tamil and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Ambareeshkumar/BERT-Tamil and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="./metaphor_classifier_tamil/",
    num_train_epochs=6,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.09,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps = 0
)

In [20]:
# pip install optuna

In [21]:
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
# import optuna
# from transformers import Trainer, TrainingArguments

# def objective(trial):
#     output_dir = "./metaphor_detection/"
#     num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
#     per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16])
#     warmup_steps = trial.suggest_int("warmup_steps", 100, 1000, step=100)
#     weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1, step=0.01)

#     training_args = TrainingArguments(
#         output_dir=output_dir,
#         num_train_epochs=num_train_epochs,
#         per_device_train_batch_size=per_device_train_batch_size,
#         per_device_eval_batch_size=16,
#         warmup_steps=warmup_steps,
#         weight_decay=weight_decay,
#         logging_dir='./logs',
#         logging_steps=10,
#         eval_steps=0
#     )

#     # Initialize Trainer and start training
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=dataset_train,
#         eval_dataset=dataset_eval,
#         tokenizer=tokenizer,
#         compute_metrics = compute_metrics
#     )

#     # Train the model
#     trainer.train()

#     # Return evaluation metric (e.g., accuracy) for Optuna to maximize
#     metrics = trainer.evaluate()
#     return metrics['eval_accuracy']

# # Create and run Optuna study
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

# # Get the best hyperparameters found by Optuna
# best_params = study.best_params
# print("Best Hyperparameters:", best_params)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

Step,Training Loss
10,0.730800
20,0.690800
30,0.692700
40,0.713700
50,0.661000
60,0.705700
70,0.682500
80,0.685200
90,0.720200
100,0.734400


In [ ]:
# tokenizer.push_to_hub("sarraj19/metaphor_classifier_tamil")
# trainer.push_to_hub()

In [ ]:
y_test_predict = trainer.predict(dataset_test)

# Take a look at the predictions
# y_test_predict

In [ ]:
y_test_logits = y_test_predict.predictions

# First 5 predicted probabilities
# y_test_logits[:5]

In [ ]:
# Predicted probabilities
y_test_probabilities = tf.nn.softmax(y_test_logits)

# First 5 predicted logits
# y_test_probabilities[:5]

In [ ]:
# Predicted labels
y_test_pred_labels = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted probabilities
# y_test_pred_labels[:5]

In [ ]:
# Actual labels
y_test_actual_labels = y_test_predict.label_ids

# First 5 predicted probabilities
# y_test_actual_labels[:5]

In [ ]:
trainer.evaluate(dataset_test)

In [ ]:
# Load f1 metric
metric_f1 = evaluate.load("f1")

# Compute f1 metric
metric_f1.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

In [ ]:
metric_precision = evaluate.load("precision")

# Compute recall metric
metric_precision.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)

In [ ]:
# Load recall metric
metric_recall = evaluate.load("recall")

# Compute recall metric
metric_recall.compute(predictions=y_test_pred_labels, references=y_test_actual_labels)